In [ ]:
# Импорт необходимых модулей 
import matplotlib
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Настройки для визуализации
# Если используется темная тема - лучше текст сделать белым
TEXT_COLOR = 'black'

matplotlib.rcParams['figure.figsize'] = (15, 10)
matplotlib.rcParams['text.color'] = 'black'
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['axes.labelcolor'] = TEXT_COLOR
matplotlib.rcParams['xtick.color'] = TEXT_COLOR
matplotlib.rcParams['ytick.color'] = TEXT_COLOR

# Зафиксируем состояние случайных чисел
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Логистическая регрессия

К вам когда-нибудь приходила делегация ботаников (не обидное слово, а уважаемые ученые!) и умоляла помочь им автоматизировать процесс определения типов ирисов? Давайте представим, что это случилось недавно, а что самое важное для нас, у них есть данные, которые отражают, как сейчас ставятся типы по измерению размеров ирисов! Закатываем рукава, давайте поможем им, ведь мы уже знаем, что такое задача классификации и как с ней работать!


# Анализ и предобработка

Для работы над задачей классификации будет использован Iris Dataset https://scikit-learn.org/stable/datasets/index.html#iris-dataset. Данный датасет загружается функцией `sklearn.datasets.load_iris()`. 

Из описания на сайте мы определяем основные ключи в объекте `sklearn.utils.Bunch`:
- `DESCR` - строчное описание датасета;
- `data` - данные с признаками;
- `feature_names` - названия признаков;
- `target_names` -  названия классов ирисов;
- `targets` - целочисленные индексы классов.

In [ ]:
from sklearn.datasets import load_iris

iris_data = load_iris()

In [ ]:
print(iris_data['DESCR'])

In [ ]:
print(iris_data['target_names'])

Из описания видно, что в наборе данных четыре признака, каждый признак представлен в единицах измерения [см] - это значит, что все признаки числовые (вещественные). Целевыми классами являются три разновидности ирисов:
- setosa
- versicolor
- virginica

![Замещающий текст](https://sundeeppothula1993.github.io/ARTML//assets/img/iris.png)

Уже на данном этапе мы можем сделать важное заключение: так как типов ирисов три и каждый из них исключает остальные (то есть цветку можно присвоить лишь один из типов), то задача, над которой будем работать имеет тип мультиклассовой классификации.

Начнем анализ данных с загрузки данных в `pandas` в формате таблицы, в которую целевые классы запишем названиями в колонку `species`.

In [ ]:
feature_names = iris_data['feature_names']
df = pd.DataFrame(iris_data['data'], columns=feature_names)
species_names = iris_data['target_names']
target_idxs = iris_data['target']
df['species'] = species_names[target_idxs]

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.groupby('species').size()

По основной информации видно, что данные не имеют пропусков, всего имеется 150 записей (соответствует описанию), а также, что очень важно, данные **равномерно распределены по классам** - каждый класс содержит по 50 записей. 

После валидации информации на соответсвие описанию произведем анализ распределения данных.

Для решения задачи классификации требуется как можно более однозначно отнести данные к одному из предсказываемых классов. Для анализа данных воспользуемся двумя полезными функциями визуализации `seaborn.boxplot()` и `seaborn.violinplot()`. Обе эти функции показывают распределение переменной относительно конкретного класса и позволяют произвести унивариатный анализ (каждый признак рассматривается отдельно).

In [ ]:
fig, axs = plt.subplots(2, 2)

for i, feat_name in enumerate(feature_names):
    row = i//2
    col = i%2
    sns.boxplot(x = 'species', y = feat_name, data = df, order = species_names, ax = axs[row, col]);
    
fig.tight_layout();

In [ ]:
fig, axs = plt.subplots(2, 2)

for i, feat_name in enumerate(feature_names):
    row = i//2
    col = i%2
    sns.violinplot(x = 'species', y = feat_name, data = df, order = species_names, ax = axs[row, col]);
    
fig.tight_layout();

Boxplot представление называется "Ящик с усами", которая показывает не только основные характеристики распределения (медиана, квартили, выбросы), но и соотношение между отдельными классами:
![Картинка](https://upload.wikimedia.org/wikipedia/commons/3/32/Densityvsbox.png)

График скрипки показывает симметричное распределение данных с учетом классов. Тут нет никаких обозначений, по сути мы отображает kde распределение, но только симметрично с двух сторон.

Посмотрим на графики как скрипки, так и ящика с усами. Мы можем заметить, что класс `setosa` хорошо отделим от остальных классов по признакам `petal length` и `petal width` - представьте, что мы можем провести горизонтальную линию, так, что она разделит класс `setosa` от остальных. По факту, нам даже никакую модель строить не надо было, если бы задачей было бы классифицировать, является ли ирис классом `setosa` или остальными (бинарная классификация) - хватит порогового значения по одному из признаков! Но тут нужно разделить на три класса, а другие два класса не получится так просто разделить.

Признаки `sepal *` не позволяют явно разделить классы, пересечения классов явно больше, чем пересечение по признакам `petal *`, что делает последние признаки более перспективными в использовании для создания модели классификации.

Помимо независимого анализа одиночных признаков, полезно также выявить взаимное влияние признаков на возможность разделения классов.

В данном случае небольшое количество признаков позволяет использовать попарное отображение признаков с помощью функции `seaborn.pairplot()`.

In [ ]:
sns.pairplot(df, hue='species')

Изображения на главной диагонали показывают распределения самих признаков, остальные ячейки демонстрируют распределение данных по двум признакам. Использование данной визуализации данных позволяет не только провести оценку возможности разделения классов по двум признакам, но также выявить возможную корреляцию признаков между собой.

Также мы можем видеть, что на одномерных графиках распределения классы `versicolor` и `virginica` сильно пересекались, но при этом мы видим, что при использовании нескольких признаков (например, `petal_width` и `sepal_width`), группы данных уже проще разделить, а значит и наша модель сможет справиться!

В данном случае сильно коррелирующими признаками выглядят признаки `petal length` и `petal width`, что легко проверить с помощью визуализации коэффициентов корреляции.

# Разработка модели

Теперь пора делать модель для мультиклассовой логистической регрессии.

In [ ]:
from sklearn.model_selection import train_test_split

TRAIN_RATIO = 0.7

X = df[feature_names]
y = target_idxs

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    train_size=TRAIN_RATIO, 
    random_state=RANDOM_STATE,
    stratify=y
)

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

После разбиения на выборки создадим объект модели. Обратите внимание, в аргументах задается метод `multinomial`, который определяет решаемую задачу, а также задается состояние генератора случайных чисел для фиксации и сохранения повторяемости.

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(
    random_state=RANDOM_STATE,
    max_iter=200,
    multi_class='multinomial'
)
logreg.fit(X_train, y_train)

Для ознакомления с результатами работы модели возьмем первый пример из тестовой выборки и проверим работу доступных у классификационной модели методов:
- `predict()` - методы выполнения предсказания, сразу выдает результат (индекс класса);
- `predict_proba()` - медот выполнения предсказания, при этом результат представляется в виде конечных вероятностей по классам (для *multinominal* результат Softmax, для *ovr* - Sigmoid по каждому классу). 
- `predict_log_proba()` - тоже самое, что и `predict_proba()`, но значения обработаны функцией логарифма.

> Обратите внимание, что все методы `predict*()` ожидают на вход 2D массив. Методы `predict_*proba()` выдают результатом 2D массив.

In [ ]:
sample = X_test.iloc[0]

prediction = logreg.predict([sample])
predict_proba = logreg.predict_proba([sample])
predict_log_proba = logreg.predict_log_proba([sample])

print(f'  Sample:\n{sample}')
print(f'  Prediction proba:\n{predict_proba[0]}')
print(f'  Prediction proba sum:\n{sum(predict_proba[0])}')
print(f'  Prediction log proba:\n{predict_log_proba[0]}')
print(f'  Prediction:\n{prediction}')
print(f'  Prediction name:\n{species_names[prediction]}')

Как видно, для мультиноминальной классификации выполняется условие равенства суммы элементов единице (с точность до округления), что является подтверждением использования функции Softmax.

Помимо основных методов, объект логистической регрессии также имеет аттрибуты, хранящие значение весов и смещения.

In [ ]:
print(f'  Classes: {logreg.classes_}')
print(f'  Weights:\n{logreg.coef_}')
print(f'  Bias:\n{logreg.intercept_}')

После ознакомления с базовыми методами и аттрибутами самое время оценить работу классификатора. Для этого воспользуемся базовыми методами визуализации и числовой оценки.

In [ ]:
from sklearn.metrics import plot_confusion_matrix

disp = plot_confusion_matrix(
    logreg, X_test, y_test,
    display_labels=species_names)

Данный график является матрицей замешательств (confusion matrix ~ CM). Он демонстрирует то, сколько примеров, отмеченных определенным классом было предсказано верно, а сколько нет. Это позволяет оценить, какие классы модель предсказывает неверно. По горизонтали рамещаются классы, которые являются разметкой (Ground-Truth/True), по вертикали отмечаются классы предсказанные (Predicted).

В данном случае модель верно предсказала все примеры по классу `setopia` (как мы видели, данный класс легко отделим от других классов). Неверно предсказан пример, являющийся классом `versicolor` (он был отмечен классом `virginica`). Также, произошли два промаха по классу `virginica` - отмечены как `versicolor`.

> Мы также видели по распределению, что классы, в которых происходит путаница сложнее отделяются, в них и произошли ошибки.

Помимо матричного представления полезно также оценить основные числовые показатели.

In [ ]:
from sklearn.metrics import classification_report

y_pred = logreg.predict(X_test)
report = classification_report(
    y_test, y_pred, 
    target_names=species_names
)

print(report)

Представленный отчет показывает численные значения основных классификационных характеристик по каждому классу, а также усредненные значения по методам `macro` и `weighted`.

Данные показатели позволяют оценить характер работы модели, что позволяет получить требуемые характеристики и сравнивать различные представления моделей.

Замечательно! Вот мы и научились обучать модель, которая может производить классификацию ирисов с точностью аж 93%! Теперь пора изучить более широко ее возможности и влияние различных параметров.

# Кросс-валидация

Кросс-валидация очень полезна в случае малого набора данных - наш случай! Для этого `sklearn` имеет ряд функций - одна из них функция обучения + оценки `sklearn.model_selection.cross_val_score()`.

In [ ]:
from sklearn.model_selection import cross_val_score

X = df[feature_names]
y = target_idxs

# Данная функция используется только для оценки 
#   (она обучает модель внутри, но не возвращает ее)
scores = cross_val_score(
    logreg,     # Модель для оценки
    X,          # Данные для обучения
    y,          # Разметка для обучения
    cv=5,       # Количество фолдов
    scoring='f1_macro'  # Желаемая метрика
)
print(f'Scores: {scores}')
print(f'F1 (macro): {scores.mean(): 0.2f} (+/- {scores.std() * 2: 0.2f})')

Использование кросс-валидации позволяет произвести более обобщенную оценку по сравнению с явным выделение части набора данных. Как видно, оценка при выборке различных фолдов вырьируется очень сильно от 0.93 до 1.0. Это как раз явление, которое можно наблюдать при использовании малого количества данных.

В идеальном случае кросс-валидация используется для подбора вариантов модели на обучающих данных, при этом тестовые данные остаются нетронутыми. В данной работе предлагается использовать кросс-валидацию, как способ более полного взгляда на работу полученной модели с данными. Соответственно, рекомендуется использовать ее при оценке.

# Задание

* Проведите базовый анализ данных и разработайте базовую модель (не забудьте оценить работу модели);
* Оцените работу модели при использовании `StandartScaler`;
* Исследуйте работу модели `LogisticRegression` с изменением аргумента (3 изменения / различных значения) `iter` - объяснить, что происходит при установке значения по-умолчанию (значение взять из справки);
* Оцените работу модели при обучении на двух признаках (попробовать 2 пары признаков):
    - sepal length (cm) + sepal width (cm);
    - petal length (cm) + petal width (cm);
* Найдите лучшую пару признаков методом кросс-валидации на обучающих данных (перебрать все возможные пары);
* Освойте и изучите работу подхода классификации [**К ближайших соседей (KNN)**](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier), продемонстрируйте работу и основную суть метода, сравните с показателями логистической регрессии;
* Оцените влияние аргумента `n_neighbors` на работу модели KNN (7 различных значений). Постройте график или таблицу основных показателей метрик от значения количества соседей.
* Отобразите **плоскость решений** при использовании двух признаков как лучшей модели логистической регрессии, так и лучшей модели KNN.


# Вопросы


* Почему массив весов имеет размер 3х4, а массив смещений - 3 элемента в модели логистической регрессии?
* В чем различия методов усреднения статистики `macro`, `micro` и `weighted`?
* Что означает `support` в отчете классификации?
* Продемонстрируйте расчет показателей `recall` и `precision` одного из классов по любой из CM.
* Что происходит при использовании аргумента `stratify` при разделении на выборки? Что будет, если не использовать данный аргумент? 
* В чем разницах подходов обучения модели линейной регрессии и логистичесой регрессии?
* На основе чего работает метод KNN?
* Как влияет количество соседей на работу модели? 